### 1. Faz BM25 anual, em vez do corpus inteiro ###

In [1]:
import os

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/falando_nela_v2/src')
    print("Current working directory (Colab):", os.getcwd())
else:
  print("Current working directory (not Colab):", os.getcwd())

Mounted at /content/drive
Current working directory (Colab): /content/drive/MyDrive/falando_nela_v2/src


In [9]:
# prompt: Conecte-se a este banco de dados: /content/drive/MyDrive/falando_nela_v2/data/DiscursosSenadores.sqlite

import sqlite3
DB_PATH = '/content/drive/MyDrive/falando_nela_v2/data/DiscursosSenadores.sqlite'

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()




In [10]:
!pip install openpyxl

import sqlite3
import pandas as pd

# Consulta de discursos
query_discursos = """
SELECT
    CodigoPronunciamento,
    DataPronunciamento,
    TextoIntegral
FROM
    Discursos
"""

# Lê os dados em DataFrames
df_discursos = pd.read_sql_query(query_discursos, conn)

# Fecha conexão
conn.close()






In [36]:
!pip install rank_bm25 xlsxwriter unidecode

import os
import re
import pandas as pd
from rank_bm25 import BM25Okapi
from unidecode import unidecode

# Garante que a data está no formato datetime
df_discursos["DataPronunciamento"] = pd.to_datetime(df_discursos["DataPronunciamento"], errors="coerce")

# Lista de anos presentes
anos = sorted(df_discursos["DataPronunciamento"].dt.year.dropna().unique())

# Pasta de saída
os.makedirs("../data/analises_bm25_tfidf", exist_ok=True)

# Pré-processamento
def preprocess_text(text):
    if isinstance(text, str):
        text = unidecode(text.lower())
        text = re.sub(r"[^\w\s]", "", text)
        return text
    return ""


# Lista para guardar todos os DataFrames de scores
lista_scores = []

for ano in anos:
    print(f"\n📅 Processando BM25 para o ano {ano}")

    df_ano = df_discursos[df_discursos["DataPronunciamento"].dt.year == ano].copy()

    if df_ano.empty:
        print(f"[⚠️] Nenhum discurso em {ano}.")
        continue

    df_ano = df_ano[df_ano["TextoIntegral"].notnull()]
    df_ano["TextoProcessado"] = df_ano["TextoIntegral"].apply(preprocess_text)
    df_ano = df_ano[df_ano["TextoProcessado"].str.strip() != ""]
    if df_ano.empty:
        print(f"[⚠️] Nenhum discurso válido após preprocessamento em {ano}.")
        continue

    corpus = [doc.split() for doc in df_ano["TextoProcessado"].tolist()]
    codigos = df_ano["CodigoPronunciamento"].tolist()

    bm25 = BM25Okapi(corpus)
    scores = bm25.get_scores(["constituicao"])

    df_scores_ano = pd.DataFrame({
        "CodigoPronunciamento": codigos,
        f"BM25_{ano}": scores
    })

    lista_scores.append(df_scores_ano)

# Junta tudo por concatenação externa
df_bm25_final = pd.DataFrame({"CodigoPronunciamento": df_discursos["CodigoPronunciamento"].unique()})

for df_ano in lista_scores:
    df_bm25_final = df_bm25_final.merge(df_ano, on="CodigoPronunciamento", how="left")








📅 Processando BM25 para o ano 2007

📅 Processando BM25 para o ano 2008

📅 Processando BM25 para o ano 2009

📅 Processando BM25 para o ano 2010

📅 Processando BM25 para o ano 2011

📅 Processando BM25 para o ano 2012

📅 Processando BM25 para o ano 2013

📅 Processando BM25 para o ano 2014

📅 Processando BM25 para o ano 2015

📅 Processando BM25 para o ano 2016

📅 Processando BM25 para o ano 2017

📅 Processando BM25 para o ano 2018

📅 Processando BM25 para o ano 2019

📅 Processando BM25 para o ano 2020

📅 Processando BM25 para o ano 2021

📅 Processando BM25 para o ano 2022

📅 Processando BM25 para o ano 2023

📅 Processando BM25 para o ano 2024


In [38]:
# Cria a coluna BM25_anual
colunas_bm25 = [col for col in df_bm25_final.columns if col.startswith("BM25_2")]
df_bm25_final["BM25_anual"] = df_bm25_final[colunas_bm25].max(axis=1)

       CodigoPronunciamento  BM25_2007  BM25_2008  BM25_2009  BM25_2010  \
67344                511384        NaN        NaN        NaN        NaN   
67345                511448        NaN        NaN        NaN        NaN   
67346                511592        NaN        NaN        NaN        NaN   
67347                511650        NaN        NaN        NaN        NaN   
67348                511856        NaN        NaN        NaN        NaN   

       BM25_2011  BM25_2012  BM25_2013  BM25_2014  BM25_2015  BM25_2016  \
67344        NaN        NaN        NaN        NaN        NaN        NaN   
67345        NaN        NaN        NaN        NaN        NaN        NaN   
67346        NaN        NaN        NaN        NaN        NaN        NaN   
67347        NaN        NaN        NaN        NaN        NaN        NaN   
67348        NaN        NaN        NaN        NaN        NaN        NaN   

       BM25_2017  BM25_2018  BM25_2019  BM25_2020  BM25_2021  BM25_2022  \
67344        NaN       

In [39]:


# Remove as colunas antigas de BM25
columns_to_drop = [col for col in df_bm25_final.columns if col.startswith('BM25_') and col != 'BM25_anual']
df_bm25_final = df_bm25_final.drop(columns=columns_to_drop)

print(df_bm25_final.tail())

       CodigoPronunciamento  BM25_anual
67344                511384    2.262250
67345                511448    0.000000
67346                511592    2.467401
67347                511650    0.000000
67348                511856    0.000000


In [40]:
df_bm25_final.head()

,CodigoPronunciamento,BM25_anual
0,366328,0.000000
1,366329,1.648794
2,366330,0.000000
3,366331,2.301064
4,366332,0.000000


In [41]:

# Salva o DataFrame em um arquivo Excel usando openpyxl
df_bm25_final.to_excel("../data/analises_bm25_tfidf/bm25_por_ano_constituicao.xlsx", engine="openpyxl", index=False)

In [42]:
# prompt: transfira o arquivo produzido para a pasta ../data. Use o nome dados_para_tableau_5.xlsx.

import shutil

# Define o caminho do arquivo de origem e destino
source_path = "../data/analises_bm25_tfidf/bm25_por_ano_constituicao.xlsx"
destination_path = "../data/dados_para_tableau_5.xlsx"

# Move o arquivo
shutil.move(source_path, destination_path)

print(f"Arquivo movido de '{source_path}' para '{destination_path}'")


Arquivo movido de '../data/analises_bm25_tfidf/bm25_por_ano_constituicao.xlsx' para '../data/dados_para_tableau_5.xlsx'
